In [267]:
import pandas as pd 
import matplotlib as plt

In [172]:
# Import meta deta
meta_df = pd.read_csv("output/albania_metadata.csv")

In [173]:
meta_df.columns

Index(['variable_name', 'variable_description', 'module_name',
       'module_description', 'data_type', 'unique_id'],
      dtype='object')

In [174]:
meta_df['unique_id'].value_counts()

unique_id
id      472
hhid    346
Name: count, dtype: int64

In [175]:
#Split metadata into groups to combine to create household and individual datasets
grouped = meta_df.groupby(["unique_id"])
ind_df = grouped.get_group(("id"))
h_df = grouped.get_group(("hhid"))

C:\Users\PB\AppData\Local\Temp\ipykernel_14432\1015254879.py:3: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  ind_df = grouped.get_group(("id"))
C:\Users\PB\AppData\Local\Temp\ipykernel_14432\1015254879.py:4: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  h_df = grouped.get_group(("hhid"))


In [176]:
#Start with household data
house_mod_names = h_df['module_name'].unique().tolist()
house_mod_names

['Modul_1A_householdroster',
 'Modul_1B_householdroster',
 'Modul_3_internet',
 'Modul_5_Non-Farm_Enterprises',
 'Modul_6D_Shocks_to_the_household',
 'Modul_9B_Access_to_Health_Care',
 'Modul_10_Fertility',
 'Modul_12A_Purchases_past30_days',
 'Modul_12B_Purchases_past6_months',
 'Modul_12C_Purchases_past12_months',
 'Modul_13A_Description_of_Dwelling',
 'Modul_13B_Utilities',
 'Modul_13C1_Household_Durables',
 'Modul_13C2_Household_Durables',
 'Modul_14_Social_Protection',
 'Modul_15_Other_Income',
 'Modul_17_Identification_of_Agriculture_Hh_Q1-2',
 'Modul_17_Identification_of_Agriculture_Hh_Q3-9',
 'Modul_17_Identification_of_Agriculture_Hh_Q10-11',
 'poverty',
 'Weight_lsms2012_retro']

In [223]:
house_mod_data = {}
house_mod_desc = []

for module in house_mod_names:
    data = pd.read_spss(f"raw_data/{module}.sav") # Extract raw data from .sav file

    house_mod_desc.append({"module_name" : module, "n_variables" : data.shape[1], "n_entries" : data.shape[0]}) 
    house_mod_data[module] = data

In [213]:
house_mod_data_copy = house_mod_data.copy()
house_mod_desc_copy = house_mod_desc.copy()

In [221]:
house_mod_data = house_mod_data_copy.copy()
house_mod_desc = house_mod_desc_copy.copy()

In [224]:
 # Sanity check that mapping for hhid to psu- hh is the same across data
working_df = house_mod_data.get('Modul_1A_householdroster').copy()
working_df['hhid2'] = working_df['hhid'].astype(str) + '_' + working_df['psu'].astype(str) + '_' + working_df['hh'].astype(str)
ind0 = working_df['hhid2']

working_df = house_mod_data.get('Modul_3_internet').copy()
working_df['hhid2'] =  working_df['hhid'].astype(str) + '_' + working_df['psu'].astype(str) + '_' + working_df['hh'].astype(str)
ind1 = working_df['hhid2']

ind0.equals(ind1)


True

In [225]:
""" 
'Modul_1B_householdroster' has issues with hhid, with 18664 invalid entries out of total 25335 entries.
The approach taken is to form a master mapping between psu-hh to the hhid from the first module, which has been verified as having a consistent mapping.
This master mapping will be used to map the hhid in modul-1b to the correct hhid entry using the given fully valid psu and hh columns in modul 1b
"""
#Creating master mapping dictionary from module 1a data
working_df = house_mod_data.get('Modul_1A_householdroster').copy()
working_df['proxy_hhid'] = working_df['psu'].astype(str) + '_' + working_df['hh'].astype(str)
mapping_dict = dict(zip(working_df['proxy_hhid'], working_df['hhid'])) # Mapping function takes in proxy as input and outputs hhid 

#Amending module 1b using mapping dict
working_df = house_mod_data.get('Modul_1B_householdroster')
working_df['proxy_hhid'] = working_df['psu'].astype(str) + '_' + working_df['hh'].astype(str)
working_df['hhid'] = working_df['proxy_hhid'].map(mapping_dict)
working_df.drop(columns = ["proxy_hhid"],inplace = True) 

"""
Additional error correction is required for Module 10, with the misnamed psu, hh columns
"""
working_df = house_mod_data.get('Modul_10_Fertility')
working_df = working_df.rename(columns = {"M0_Q00":"psu","M0_Q01":"hh"},inplace = True)

In [ ]:
for name in house_mod_names:
    working_df = house_mod_data.get(name)
    if 'hhid' in working_df.columns:
        working_df.drop(columns = ["psu","hh"],inplace = True) # Drop psu and hh since redundant
    else:
        working_df['proxy_hhid'] = working_df['psu'].astype(str) + '_' + working_df['hh'].astype(str)
        working_df['hhid'] = working_df['proxy_hhid'].map(mapping_dict)
        working_df.drop(columns = ["psu","hh","proxy_hhid"],inplace = True) # Drop psu and hh since redundant
    
    print(working_df.columns)
    

Index(['idcode', 'hhid', 'm1a_q02', 'm1a_q03', 'm1a_q04', 'm1a_q05y',
       'm1a_q05m', 'm1a_q06', 'm1a_q07', 'm1a_q08', 'm1a_q09', 'm1a_q10',
       'm1a_q11', 'm1a_q12', 'm1a_q13', 'm1a_q14', 'm1a_q15', 'm1a_q16',
       'm1a_q17', 'm1a_q18', 'm1a_q19', 'm1a_q20', 'm1a_q21', 'm1a_q22'],
      dtype='object')
Index(['idcode', 'M1B_SEX', 'M1B_AGE', 'M1B_Q01', 'M1B_Q02', 'M1B_Q4B',
       'M1B_Q03', 'M1B_Q4A', 'M1B_Q5A', 'M1B_Q5B', 'M1B_Q5C', 'M1B_Q5D',
       'M1B_Q5E', 'M1B_Q5F', 'M1B_Q5G', 'hhid'],
      dtype='object')
Index(['idcode', 'M3_SEX', 'M3A_Q5', 'M3_Q01', 'M3_Q02', 'M3_Q03', 'M3_Q04',
       'M3_Q05', 'M3_Q6A', 'M3_Q6B', 'M3_Q6C', 'M3_Q6D', 'm3_q07_1',
       'm3_q07_2', 'm3_q07_3', 'm3_q07_4', 'm3_q07_5', 'm3_q07_6', 'm3_q07_7',
       'm3_q07_8', 'm3_q07_9', 'm3_q7_10', 'm3_q7_11', 'm3_q7_12', 'M3_Q08',
       'hhid'],
      dtype='object')
Index(['M5_Q03', 'M5_Q04', 'M5_Q05', 'M5_Q06', 'M5_Q07', 'M5_Q08', 'M5_Q09',
       'M5_Q10', 'M5_Q11', 'M5_Q12', 'M5_Q13', 'M5_Q14

In [240]:
house_mod_names

['Modul_1A_householdroster',
 'Modul_1B_householdroster',
 'Modul_3_internet',
 'Modul_5_Non-Farm_Enterprises',
 'Modul_6D_Shocks_to_the_household',
 'Modul_9B_Access_to_Health_Care',
 'Modul_10_Fertility',
 'Modul_12A_Purchases_past30_days',
 'Modul_12B_Purchases_past6_months',
 'Modul_12C_Purchases_past12_months',
 'Modul_13A_Description_of_Dwelling',
 'Modul_13B_Utilities',
 'Modul_13C1_Household_Durables',
 'Modul_13C2_Household_Durables',
 'Modul_14_Social_Protection',
 'Modul_15_Other_Income',
 'Modul_17_Identification_of_Agriculture_Hh_Q1-2',
 'Modul_17_Identification_of_Agriculture_Hh_Q3-9',
 'Modul_17_Identification_of_Agriculture_Hh_Q10-11',
 'poverty',
 'Weight_lsms2012_retro']

In [278]:
data = house_mod_data.get("Modul_17_Identification_of_Agriculture_Hh_Q3-9")
data['hhid'].value_counts()

hhid
55603.0    7
47602.0    7
55606.0    6
7005.0     6
7004.0     6
          ..
42004.0    1
40811.0    1
34804.0    1
34905.0    1
3904.0     1
Name: count, Length: 3028, dtype: int64

In [274]:
data.shape

(120078, 5)

In [280]:
rows = data.loc[data['hhid'] ==55606.0]
rows

,M17_Q03,M17_Q04,M17_Q05,M17_Q06,M17_Q07,M17_Q08,M17_Q9A,M17_Q9B,hhid
522,FUSHA E TRASHE,Privatized (L. 7501),1992.0,Deed (from Law 7501 in 1991),600.0,Pasture,NaN,NaN,55606.0
523,GORICA E BEBES,Privatized (L. 7501),1992.0,Deed (from Law 7501 in 1991),10500.0,Pasture,NaN,NaN,55606.0
524,KORIDHI,Privatized (L. 7501),1992.0,Deed (from Law 7501 in 1991),2600.0,Pasture,NaN,NaN,55606.0
525,OXHAKU,Privatized (L. 7501),1992.0,Deed (from Law 7501 in 1991),250.0,Pasture,NaN,NaN,55606.0
4426,VRESHTI,Privatized (L. 7501),1992.0,Deed (from Law 7501 in 1991),900.0,Tree crop land,Grape,No other,55606.0
4427,VRESHTI A,Privatized (L. 7501),1992.0,Deed (from Law 7501 in 1991),1600.0,Tree crop land,Grape,No other,55606.0


In [249]:
hhid_df = house_mod_data.get("Modul_1A_householdroster").merge(house_mod_data.get("Modul_3_internet"), how = 'left', on = 'hhid')
pd.DataFrame(hhid_df).to_csv("output/household_data.csv")
#print(house_mod_data.get("Modul_1A_householdroster").shape)
#print(house_mod_data.get("Modul_1B_householdroster").shape)

In [ ]:
hhid_df = pd.DataFrame() # Initialise empty dataframe to store merged data
pd.DataFrame(hhid_df).to_csv("output/household_data.csv")

for name in house_mod_names:
    data = house_mod_data.get(name)
    hhid_df = pd.read_pickle("output/household_data.pkl")
    if hhid_df.empty:
        hhid_df = data
        pd.DataFrame(hhid_df).to_pickle("output/household_data.pkl")
    else: 
        hhid_df = hhid_df.merge(data, on = "hhid")
        pd.DataFrame(hhid_df).to_pickle("output/household_data.pkl")
        

MemoryError: Unable to allocate 1.90 GiB for an array with shape (48, 5301018) and data type float64